<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/Rayleigh_Experiments/MNIST_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/Colab Notebooks/Rayleigh_Experiments/'
print(sys.path)

Mounted at /content/drive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/', '/content/drive/MyDrive/CertifiableBayesianInference']


In [ ]:
# Author: Matthew Wicker

# Description: Minimal working example of training and saving
# a BNN trained with Bayes by backprop (BBB)
# can handle any Keras model
# 描述：训练和保存的最小工作示例
# 通过反向传播（BBB）用贝叶斯训练的 BNN
# 可以处理任何 Keras 模型
# import sys, os
# from pathlib import Path
# path = Path(os.getcwd())
# sys.path.append(str(path.parent))

# 作者封装的BayesKeras模块
import BayesKeras
import BayesKeras.optimizers as optimizers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#tf.debugging.set_log_device_placement(True)
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 引入shell指令的参数
# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--eps")
# parser.add_argument("--lam")
# parser.add_argument("--rob")
# parser.add_argument("--gpu", nargs='?', default='0,1,2,3,4,5')
# parser.add_argument("--opt")

# args = parser.parse_args()
# eps = float(args.eps)
# lam = float(args.lam)
# optim = str(args.opt)
# rob = int(args.rob)
# gpu = str(args.gpu)
eps = 0.11
lam = 0.25
rob = 5
optim = 'SWAG'
gpu = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

# 划分训练集，测试集
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

# 构建Sequential模型
model = Sequential()
# 构建第Dense隐藏层并添加到模型中
# 添加具有512个神经元的Dense隐藏层，使用relu激活函数
model.add(Dense(512, activation="relu" #, input_shape=(1, 28*28)
))
# 添加具有10个神经元的Dense隐藏层，使用softmax激活函数
model.add(Dense(10, activation="softmax"))

inf = 2
full_covar = False
if(optim == 'VOGN'):
    # was 0.25 for a bit  
    inf = 2
    learning_rate = 0.35; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    inf = 10
    learning_rate = 0.45; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
    learning_rate = 0.1; decay=0.0
    # 优化器.随机加权平均高斯
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SWAG-FC'):
    learning_rate = 0.01; decay=0.0; full_covar=True
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SGD'):
    learning_rate = 1.0; decay=0.0
    opt = optimizers.StochasticGradientDescent()
elif(optim == 'NA'):
    inf = 2
    learning_rate = 0.001; decay=0.0
    opt = optimizers.NoisyAdam()
elif(optim == 'ADAM'):
    learning_rate = 0.00001; decay=0.0
    opt = optimizers.Adam()
elif(optim == 'HMC'):
    learning_rate = 0.075; decay=0.0; inf=250
    linear_schedule = False
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
# 编译模型以使用贝叶斯推理进行训练
if(rob == 0 or rob == 5 or rob == 6):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif(rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

bayes_model = opt.compile(model, loss_fn=loss, epochs=20, learning_rate=learning_rate,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, inflate_prior=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam) #, preload="SGD_FCN_Posterior_1")

This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
BayesKeras: Using implicit prior
BayesKeras: Detected robust training at compilation. Please ensure you have selected a robust-compatible loss


In [ ]:
# steps was 50
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

100%|██████████| 469/469 [00:41<00:00, 11.42it/s]


Epoch 1, loss: 0.366, acc: 0.945, val_loss: 0.144, val_acc: 0.957, rob: 0.890, (eps = 0.000100)


100%|██████████| 469/469 [00:28<00:00, 16.23it/s]


Epoch 2, loss: 0.154, acc: 0.959, val_loss: 0.139, val_acc: 0.962, rob: 0.901, (eps = 0.001102)


100%|██████████| 469/469 [00:29<00:00, 16.15it/s]


...

Epoch 21, loss: 0.246, acc: 0.976, val_loss: 0.224, val_acc: 0.971, rob: 0.933, (eps = 0.020150)


100%|██████████| 469/469 [00:28<00:00, 16.54it/s]


Epoch 22, loss: 0.250, acc: 0.976, val_loss: 0.224, val_acc: 0.971, rob: 0.933, (eps = 0.021152)


100%|██████████| 469/469 [00:30<00:00, 15.30it/s]


Epoch 23, loss: 0.239, acc: 0.976, val_loss: 0.232, val_acc: 0.972, rob: 0.933, (eps = 0.022155)


In [ ]:
# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_FCN_Posterior_%s"%(optim, rob))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
!ls

sample_data
